In [1]:
import numpy as np

class Graph:
    def __init__(self):
        self.nodes = set()
        self.edges = set()

    def add_node(self, a):
        self.nodes.add(a)
    
    def add_edge(self, a, b):
        self.add_node(a)
        self.add_node(b)
        self.edges.add((min(a, b), max(a, b)))


class Point:
    def __init__(self, x: int, y: int, z: int):
        self.x = x
        self.y = y
        self.z = z

    def __str__(self):
        return f"({self.x}, {self.y}, {self.z})"

    def __repr__(self):
        return self.__str__()

class ImproperCube:
    def __init__(self, N: int):
        self.N = N
        self.proper = True
        self.cube = np.zeros((N, N, N), dtype=int)
        for i in range(N):
            for j in range(N):
                self.cube[i,j,(i+j)%N] = 1
                
    def __str__(self):
        return str(self.proper) + str(self.cube)

    def __repr__(self):
        return self.__str__()

    def ones(self, x: int, y: int, z: int):
        assert(x == -1 or y == -1 or z == -1)

        indices = []
        if x == -1:
            for i in range(self.N):
                if self.cube[i,y,z] == 1:
                    indices.append(i)
        elif y == -1:
            for i in range(self.N):
                if self.cube[x,i,z] == 1:
                    indices.append(i)
        elif z == -1:
            for i in range(self.N):
                if self.cube[x,y,i] == 1:
                    indices.append(i)
        else:
            assert False    
        return indices

    def move(self, a: Point, b: Point):
        LScopy = ImproperCube(N=self.N)    
        LScopy.cube = self.cube.copy()

        assert LScopy.cube[a.x,a.y,a.z] in [-1, 0]
        
        LScopy.cube[a.x,a.y,a.z] += 1
        LScopy.cube[a.x,a.y,b.z] -= 1
        LScopy.cube[a.x,b.y,a.z] -= 1
        LScopy.cube[b.x,a.y,a.z] -= 1
        LScopy.cube[a.x,b.y,b.z] += 1
        LScopy.cube[b.x,a.y,b.z] += 1
        LScopy.cube[b.x,b.y,a.z] += 1
        LScopy.cube[b.x,b.y,b.z] -= 1
        if LScopy.cube[b.x,b.y,b.z] == -1:
            LScopy.proper = False
        else:
            LScopy.proper = True

        return LScopy

    def adjacent(self):
        adj = []
        if self.proper:
            for x in range(self.N):
                for y in range(self.N):
                    for z in range(self.N):
                        if self.cube[x,y,z] == 0:
                            x_ = self.ones(-1, y, z)[0]
                            y_ = self.ones(x, -1, z)[0]
                            z_ = self.ones(x, y, -1)[0]
                            adj.append(self.move(Point(x, y, z), Point(x_, y_, z_)))
        else:
            minusone = None
            for x in range(self.N):
                for y in range(self.N):
                    for z in range(self.N):
                        if self.cube[x,y,z] == -1:
                            minusone = Point(x, y, z)
            X = self.ones(-1, minusone.y, minusone.z)
            Y = self.ones(minusone.x, -1, minusone.z)
            Z = self.ones(minusone.x, minusone.y, -1)
            # print(minusone,X,Y,Z)
            for i in range(2):
                for j in range(2):
                    for k in range(2):
                        x_ = X[i]
                        y_ = Y[j]
                        z_ = Z[k]
                        adj.append(self.move(minusone, Point(x_, y_, z_)))
        return adj

    

In [2]:
import sys
sys.setrecursionlimit(1000000000)

In [3]:
N = 4

In [4]:

LS = ImproperCube(N=N)
LS.cube

array([[[1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]],

       [[0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1],
        [1, 0, 0, 0]],

       [[0, 0, 1, 0],
        [0, 0, 0, 1],
        [1, 0, 0, 0],
        [0, 1, 0, 0]],

       [[0, 0, 0, 1],
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, 0]]])

In [5]:
# LS.adjacent()

In [6]:
def dfs(visited, u):
    visited[str(u)] = u
    for v in u.adjacent():
        if str(v) not in visited:
            dfs(visited, v)

In [7]:
visited = dict()
dfs(visited, LS)

In [8]:
# visited.values()

In [9]:
G = Graph()
for u in visited.values():
    for v in u.adjacent():
        G.add_edge(str(u), str(v))

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

# Create a graph
nodes = list(G.nodes)
nxG = nx.Graph()

# Add nodes

nxG.add_nodes_from(nodes)

node_colors = ['red' if u[0] == 'F' else 'blue' for u in nodes]

# Add edges
nxG.add_edges_from(list(G.edges))

print(f"N={N}, |nodes|={len(nodes)}, diam={nx.diameter(nxG)}")


# pos = nx.spring_layout(nxG, iterations=1000)
# # pos = nx.spring_layout(nxG)
# # Draw the graph
# nx.draw(nxG, pos, with_labels=False, node_color=node_colors, node_size=100, font_size=10, alpha=1)
# plt.show()
